<a href="https://colab.research.google.com/github/anaptoro/git_geotec2/blob/main/Aula6_e_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture 
# O comando capture é utilizado para que os outputs da célula não sejam exibidos, se quiser exibir, é só comentar a linha
!pip install geemap # pip é o comando de instalação no python, quando vc coloca a ! na frente do comando, vc está executando como se fosse no prompt de comando

In [2]:
%%capture
!pip install geopandas

In [3]:
# import geemap #aqui vc está importando a biblioteca
import ee

In [4]:
import geemap
import geopandas as gpd

In [5]:
Map = geemap.Map() #esse comando vai exigir uma autenticação

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=AmzppkfBe4hBbCiIz6jqNNf7nmyP724veOvpLS5RoOI&tc=g84tgdJQ1UToHsaS-4HLT8AZViGFVybmw7c7WA77hOQ&cc=_FSBxtW2Bt-AoqZDySf8_F6bwLUYd1XKehdSjHfh3wg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJp419rBSNLN0qcOBGzio4tBhoHbx9l8ROgjzj2sUXhZvQXLBj2L3ao

Successfully saved authorization token.


In [6]:
AE = ee.FeatureCollection('users/pca/ROI_Campina')

In [7]:
#carregando as duas classificações
RF1 = ee.Image('users/pca/RF1')
RF2 = ee.Image('users/pca/RF2')

In [8]:
lista_new=[10,20,30]
lista_raw=[1,2,3]
# //Reclassificando novamente para retornar classes para 0 até o número da última classe (4)
M2004 = RF1.remap(lista_raw,lista_new,0,'classification')# 3 classes

M2016 = RF2.remap(lista_raw,lista_new,0,'classification')#// 3 classes


In [9]:
igbpPalette = [
  'aec3d4', # nativa
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # pasto
  # '111149', # wetlands
  # 'cdb33b', # croplands
  # 'cc0013', # urban
  # '33280d', # crop mosaic
  # 'd7cdcc', # snow and ice
  # 'f7e084', # barren
  # '6f6f6f'  # tundra
]

Map = geemap.Map()
Map.addLayer(M2004,{'palette': igbpPalette, 'min': 10, 'max': 30},'M2004')
Map.addLayer(M2016,{'palette': igbpPalette, 'min': 10, 'max': 30},'M2016')
Map.centerObject(AE, 13)
Map

Map(center=[-21.61922395993749, -51.88686153437356], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
#Para realizar a diferença entre os mapas precisamos reclassificar os valores de um dos mapas
RM2016 = M2016.remap([10,20,30],[100,200,300], 0, 'remapped');    
Map.addLayer(M2004,{'palette': igbpPalette, 'min': 10, 'max': 30},'M2004')
Map.addLayer(RM2016, { 'palette': igbpPalette, 'min': 100, 'max': 300,}, 'RM2016')
Map

Map(center=[-21.61922395993749, -51.88686153437356], controls=(WidgetControl(options=['position', 'transparent…

In [11]:
# //Listar valores das diferentes classes para L5 e L8;
# //Listar valores das diferentes classes para L5 e L8;
classe0_L5 = 10 #Nativa
classe0_L8 = 100 #Nativa
classe1_L5 = 20 #umido
classe1_L8 = 200 #umido
classe2_L5 = 30 #ag
classe2_L8 = 300 #ag
# classe3_L5 = 3 #anual
# classe3_L8 = 400 #anual
# classe4_L5 = 4 #pastagem
# classe4_L8 = 500 #pastagem

In [12]:
#Achando o valor da diferença das classes no mapa
dif_class = classe0_L8-classe1_L5 #;//o que é agricultura anual em 2016 e era pastagem em 2004
print("Pastagem --> Agricultura Anual:", dif_class);
same_class = classe0_L8-classe0_L5 #;//o que permaneceu como agricultura anual entre 2004 e 2016
print("Agricultura Anual --> Agricultura Anual:", same_class);

Pastagem --> Agricultura Anual: 80
Agricultura Anual --> Agricultura Anual: 90


In [13]:
#Calculando a diferença entre os mapas do ano inicial e do ano final
diff = RM2016.subtract(M2004)

In [14]:
#Criação de raster de mudança de uso da terra de acordo com a classe de interesse

# Change = Change.updateMask(Change.eq(99))
Change = diff
Change = Change.updateMask(diff.eq(80)) #;//Nesse caso, o valor 396 é referente a mudança Pastagem --> Anual. Colocar o valor da sua classe de 

In [15]:
Map = geemap.Map()
Map.addLayer(Change, {},'Mudança')
Map.centerObject(AE, 13)
Map

Map(center=[-21.61922395993749, -51.88686153437356], controls=(WidgetControl(options=['position', 'transparent…

In [27]:
#Criação de raster de não mudança de uso da terra de acordo com a classe de interesse
nchange = diff.updateMask(diff.eq(90))#; //Nesse caso, o valor 90 significa que em 2006 era Anual e em 2016 também era Anual
Map.addLayer(nchange,{'palette':['green']},'Não Mudança')
Map

Map(bottom=1177917.0, center=[-21.619292314417528, -51.88688278198243], controls=(WidgetControl(options=['posi…

In [17]:
print(Change.getInfo())

{'type': 'Image', 'bands': [{'id': 'remapped', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -30, 'max': 300}, 'dimensions': [364, 281], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -51.93591848831331, 0, -0.00026949458523585647, -21.581395880272623]}]}


### Aula 27/10/2022 - Cálculo área de mudança

In [18]:
a = ee.Geometry(AE.geometry())
area = a.area().divide(1000 * 1000).getInfo()
print('area',area)

area 83.82724564783356


In [21]:
waterArea = Change.multiply(a).rename('waterArea')

In [28]:
rem = nchange.select('remapped')

In [30]:
area = ee.Image.pixelArea().addBands(rem)

In [40]:
hist = Change.select('remapped').reduceRegion(**{
      'reducer': ee.Reducer.autoHistogram(),
      'geometry': AE.geometry(), #need to be a geometry
      'maxPixels': 1e12
})
dic = hist.getInfo()['remapped']
codes = [ dic[i][0] for i in range(len(dic))] 

In [45]:
import pandas as pd 

columns=['code', 'n_pixel']
row_list = []
for code in codes:
    code = int(code)
    mask = Change.select('remapped').eq(code)
    mask_surface = mask.multiply(ee.Image.pixelArea())
    stats = mask_surface.reduceRegion(**{
        'reducer': ee.Reducer.count(),
        'geometry': AE.geometry(),
        'maxPixels': 1e13
    })

    row_list.append({
        columns[0]: code, 
        columns[1]: stats.getInfo()['remapped']
    })

    
hist_change = pd.DataFrame(row_list)

In [46]:
hist_change

,code,n_pixel
0,80,2991


In [47]:
hist_nchange = nchange.select('remapped').reduceRegion(**{
      'reducer': ee.Reducer.autoHistogram(),
      'geometry': AE.geometry(), #need to be a geometry
      'maxPixels': 1e12
})
dic = hist_nchange.getInfo()['remapped']
codes = [ dic[i][0] for i in range(len(dic))] 

In [48]:


columns=['code', 'n_pixel']
row_list = []
for code in codes:
    code = int(code)
    mask = nchange.select('remapped').eq(code)
    mask_surface = mask.multiply(ee.Image.pixelArea())
    stats = mask_surface.reduceRegion(**{
        'reducer': ee.Reducer.count(),
        'geometry': AE.geometry(),
        'maxPixels': 1e13
    })

    row_list.append({
        columns[0]: code, 
        columns[1]: stats.getInfo()['remapped']
    })

    
hist_nchange = pd.DataFrame(row_list)

In [49]:
hist_nchange

,code,n_pixel
0,90,11483


//------------- Gerar pontos aleatórios para a análise de mudança-----------------//
// Criar um vetor de pontos centrais com base no raster de diferença entre os mapas

In [50]:
class_diff=diff.reduceToVectors(scale= 30,
  geometryType= 'centroid',
  eightConnected= False,
  bestEffort= True,
  maxPixels= 1e15,
  labelProperty= 'remapped'
)


In [51]:
Map.addLayer(class_diff, {},'classdiff')
Map

Map(bottom=1177917.0, center=[-21.619292314417528, -51.88688278198243], controls=(WidgetControl(options=['posi…

In [52]:
gdf = geemap.ee_to_geopandas(class_diff, selectors=['count', 'remapped','coordinates'])
gdf.remapped.value_counts()
# gdf.head()

170    1088
280    1053
180     593
80      405
270     345
70      245
90      180
290     146
190     135
Name: remapped, dtype: int64

In [53]:
stratified = gdf.groupby('remapped').apply(lambda x: x.sample(10)).reset_index(drop=True)

In [54]:
stratified.to_file('/content/sample_data/strat.shp')

### Prática no GEE - 26/10  a  2/11 - Análise de mudança 

In [ ]:
# //* Para visualizar o mapa com todas as classes precisamos fazer mais uma reclassificação para termos valores de classes sequenciais

# //Valores das classes da variável diff
# var oldclass= [96,97,98,99,100,196,197,198,199,200,296,297,298,299,300,396,397,398,399,400,496,497,498,499,500];

# // Novos valores para as classes
# //*Obs1: É importante escrever todas as classes na forma de uma tabela para sabermos qual seriam os valores e nomes de cada classes de mudança que temos
# //*Obs2: No nosso caso, o valor de todas as classes com mudança relacionada à água, o qual o mais provável de acontecer foi um erro de classificação (ex. mudança da classes Nativa para a Classe Água), foram renomeadas para o valor '-1'
# //*Obs3: Todas as classes onde não houve mudança (ex. Nativa-Nativa, Solo-Solo, etc) foram renomeadas com o valor '0'.
# //*Obs4: No nosso caso, no final teremos valores de classes: -1,0,1,2,3,4,5,6,7,8,9,10,11 e 12. Um total de 14 classes.
# //*Obs5: É necessário termos novas em alguma ordem sequencial para poder visualizar todas elas como mapa no GEE

newclass= [1,2,3,-1,0,-1,-1,-1,0,-1,4,5,0,-1,6,7,0,8,-1,9,0,10,11,-1,12];


# //Reclassificação da variável diff com os novos valores de classe
diff_reclass= diff.remap(oldclass,newclass, null);

# //Definição da paleta de cores para as novas classes (no nosso caso, 14 cores diferentes porque temos 14 valores de classes diferentes)
palette14= ['#080200','#ccd3d6','#038735','#86c8c3','#96d672','#eefb61','#d62392','#c29057','#ffb399','#972b8c','#52ff2d','#3a7b79','#8e36d6','#2966ff']; 

# //Visualizar o mapa de mudança com as novas classes
Map.addLayer(diff_reclass, {min: -1, max: 12, palette:palette14}, 'Diff Reclass', false); // mudar os valores de min e max de acordo com a nova sequência de classes gerada

// Adicionar a legenda personalizada
